In [17]:
from docx import Document
from pandas import DataFrame, concat
import pandas as pd
import re
import os
import sqlite3

In [3]:
def read_docx(file_path):
    doc = Document(file_path)
    return doc

In [4]:
def read_docx(file_path):
    doc = Document(file_path)
    full_text = []
    for para in doc.paragraphs:
        full_text.append(para.text)
    return "\n".join(full_text)


In [12]:
def df_find_cases(file_path):
    words_for_table = []  # Use a list to store repeated entries
    sujeito_list = [] # Initialize sujeito_list outside the loop
    curso_list = []  # Initialize curso_list outside the loop
    sexo_list = [] # Initialize sexo_list outside the loop
    ano_list = [] # Initialize ano_list outside the loop
    cidade_list = [] # Initialize cidade_list outside the loop

    # Iterate through all files in the specified directory
    for filename in os.listdir(directory_path):
        if filename.endswith(".docx"):
            file_path = os.path.join(directory_path, filename)
            text = read_docx(file_path)

            ### WORDS WITH PATTERN ###
            pattern = r'(\S*\[[^\]]*s[^\]]*\][\?|\.|\,|\!\…]?)\s*(\S*)'
            matches = re.findall(pattern, text)
            words_for_table.extend(matches)

            ###### SUJEITOS #######
            sujeito_match = re.search(r'\bSUJEITO:\s*(\S+)', text)
            if sujeito_match:
                sujeito_list.extend([sujeito_match.group(1)] * len(matches))
            else:
                sujeito_list.extend(['Not Specified'] * len(matches))
            
             ##### CURSO #########
            curso_match = re.search(r'\bCURSO:\s*(\S+)', text)
            if curso_match:
                curso_list.extend([curso_match.group(1)] * len(matches))
            else:
                curso_list.extend(['Not Specified'] * len(matches))

             ##### SEXO #######
            sexo_match = re.search(r'\bSEXO:\s*(\S+)', text)
            if sexo_match:
                sexo_list.extend([sexo_match.group(1)] * len(matches))
            else:
                sexo_list.extend(['Not Specified'] * len(matches))

            
             ####### ANO ########
            ano_match = re.search(r'\bANO:\s*(\S+)', text)
            if ano_match:
                ano_list.extend([ano_match.group(1)] * len(matches))
            else:
                ano_list.extend(['Not Specified'] * len(matches))

            ####### CIDADE ########
            cidade_match = re.search(r'\bCIDADE:\s*(\S+)', text)
            if cidade_match:
                cidade_list.extend([cidade_match.group(1)] * len(matches))
            else:
                cidade_list.extend(['Not Specified'] * len(matches))
    df1 = DataFrame(words_for_table, columns=['palavra','palavra_depois'])
    df2 = DataFrame({'sujeito': sujeito_list, 'curso': curso_list, 'sexo': sexo_list, 'ano': ano_list, 'cidade': cidade_list})
    df = pd.merge(df1, df2, how='inner', left_index=True, right_index=True)


    #remove commas and periods at the end of values
    punctuation_pattern = r'[.,…!?]+$'
    a_pattern = r'-a'
    o_pattern = r'-o'
    pattern = punctuation_pattern + '|' + a_pattern + '|' + o_pattern
    df['palavra'] = df['palavra'].str.replace(pattern, '', regex=True)
    df['palavra_depois'] = df['palavra_depois'].str.replace(pattern, '', regex=True)
        
            
    return df

directory_path = "../data/"
df = df_find_cases(directory_path)
df

,palavra,palavra_depois,sujeito,curso,sexo,ano,cidade
0,sere[s],humano[∅s],SUJEITO_6,PEDAGOGIA,MULHER,PRIMEIRO,LONDRINA
1,criança[s],mas,SUJEITO_6,PEDAGOGIA,MULHER,PRIMEIRO,LONDRINA
2,a[s],minha[∅s],SUJEITO_6,PEDAGOGIA,MULHER,PRIMEIRO,LONDRINA
3,expecta[tʃɪ]va[∅s],no,SUJEITO_6,PEDAGOGIA,MULHER,PRIMEIRO,LONDRINA
4,expecta[tʃɪ]va[s],Eu,SUJEITO_6,PEDAGOGIA,MULHER,PRIMEIRO,LONDRINA
...,...,...,...,...,...,...,...
781,Fiéi[s],ao,SUJEITO_1,DIREITO,HOMEM,PRIMEIRO,OUTRA
782,felize[s],e,SUJEITO_1,DIREITO,HOMEM,PRIMEIRO,OUTRA
783,fo[ɹ(n)][tʃɪ][s],como,SUJEITO_1,DIREITO,HOMEM,PRIMEIRO,OUTRA
784,leõe[s],o[s],SUJEITO_1,DIREITO,HOMEM,PRIMEIRO,OUTRA


In [19]:
def create_apagamento_plural(row):
    if row['palavra'].endswith('[s]'):
            return 'Não'
    elif row['palavra'].endswith('[∅s]'):
            return 'Sim'

    return None


In [20]:
df['apagamento_plural'] = df.apply(create_apagamento_plural, axis=1)
df

,palavra,palavra_depois,sujeito,curso,sexo,ano,cidade,marca_de_plural,apagamento_plural
0,sere[s],humano[∅s],SUJEITO_6,PEDAGOGIA,MULHER,PRIMEIRO,LONDRINA,caso_1,Não
1,criança[s],mas,SUJEITO_6,PEDAGOGIA,MULHER,PRIMEIRO,LONDRINA,caso_1,Não
2,a[s],minha[∅s],SUJEITO_6,PEDAGOGIA,MULHER,PRIMEIRO,LONDRINA,caso_1,Não
3,expecta[tʃɪ]va[∅s],no,SUJEITO_6,PEDAGOGIA,MULHER,PRIMEIRO,LONDRINA,caso_2,Sim
4,expecta[tʃɪ]va[s],Eu,SUJEITO_6,PEDAGOGIA,MULHER,PRIMEIRO,LONDRINA,caso_1,Não
...,...,...,...,...,...,...,...,...,...
781,Fiéi[s],ao,SUJEITO_1,DIREITO,HOMEM,PRIMEIRO,OUTRA,caso_1,Não
782,felize[s],e,SUJEITO_1,DIREITO,HOMEM,PRIMEIRO,OUTRA,caso_1,Não
783,fo[ɹ(n)][tʃɪ][s],como,SUJEITO_1,DIREITO,HOMEM,PRIMEIRO,OUTRA,caso_1,Não
784,leõe[s],o[s],SUJEITO_1,DIREITO,HOMEM,PRIMEIRO,OUTRA,caso_1,Não


In [13]:
def create_marca_de_plural(row):
      if row['palavra'].endswith('[s]') and not row['palavra_depois'].startswith('s'):
            return 'caso_1'
      elif row['palavra'].endswith('[∅s]') and not row['palavra_depois'].startswith('s'):
            return 'caso_2'
      elif row['palavra'].endswith('[s]') and row['palavra_depois'].startswith('s'):
            return 'caso_3'
      elif row['palavra'].endswith('[∅s]') and row['palavra_depois'].startswith('s'):
            return 'caso_4'
       
      return None   

In [22]:
df['marca_de_plural'] = df.apply(create_marca_de_plural, axis=1)
    

In [23]:
df

,palavra,palavra_depois,sujeito,curso,sexo,ano,cidade,marca_de_plural,apagamento_plural
0,sere[s],humano[∅s],SUJEITO_6,PEDAGOGIA,MULHER,PRIMEIRO,LONDRINA,caso_1,Não
1,criança[s],mas,SUJEITO_6,PEDAGOGIA,MULHER,PRIMEIRO,LONDRINA,caso_1,Não
2,a[s],minha[∅s],SUJEITO_6,PEDAGOGIA,MULHER,PRIMEIRO,LONDRINA,caso_1,Não
3,expecta[tʃɪ]va[∅s],no,SUJEITO_6,PEDAGOGIA,MULHER,PRIMEIRO,LONDRINA,caso_2,Sim
4,expecta[tʃɪ]va[s],Eu,SUJEITO_6,PEDAGOGIA,MULHER,PRIMEIRO,LONDRINA,caso_1,Não
...,...,...,...,...,...,...,...,...,...
781,Fiéi[s],ao,SUJEITO_1,DIREITO,HOMEM,PRIMEIRO,OUTRA,caso_1,Não
782,felize[s],e,SUJEITO_1,DIREITO,HOMEM,PRIMEIRO,OUTRA,caso_1,Não
783,fo[ɹ(n)][tʃɪ][s],como,SUJEITO_1,DIREITO,HOMEM,PRIMEIRO,OUTRA,caso_1,Não
784,leõe[s],o[s],SUJEITO_1,DIREITO,HOMEM,PRIMEIRO,OUTRA,caso_1,Não


In [21]:
# Create a SQLite in-memory database
conn = sqlite3.connect(':memory:')
# Write the DataFrame to the SQLite database
df.to_sql('my_table', conn, index=False, if_exists='replace') 
query = "SELECT * FROM my_table WHERE apagamento_plural NOT IN ('Sim', 'Não');"
result = pd.read_sql_query(query, conn)
result

,palavra,palavra_depois,sujeito,curso,sexo,ano,cidade,marca_de_plural,apagamento_plural


In [24]:
frequency_table = df['apagamento_plural'].value_counts()
percentage_table = df['apagamento_plural'].value_counts(normalize=True) * 100

# Combine counts and percentages into a single DataFrame
result_df = pd.DataFrame({'Contagem': frequency_table, 'Porcentagem': percentage_table})
result_df

,Contagem,Porcentagem
apagamento_plural,,
Não,699,88.931298
Sim,87,11.068702


In [28]:
cross_table = pd.crosstab(df['sexo'], df['apagamento_plural'])
df_percentages = cross_table.copy()
total = (cross_table['Sim'] + cross_table['Não'])
df_percentages['S/ APAGAMENTO (%)'] = (cross_table['Não']/total) * 100
df_percentages['C/ APAGAMENTO (%)'] = (cross_table['Sim'] / total) * 100
df_percentages

apagamento_plural,Não,Sim,S/ APAGAMENTO (%),C/ APAGAMENTO (%)
sexo,,,,
HOMEM,332,45,88.063660,11.936340
MULHER,367,42,89.731051,10.268949


In [29]:
cross_table = pd.crosstab(df['curso'], df['apagamento_plural'])
df_percentages = cross_table.copy()
total = (cross_table['Sim'] + cross_table['Não'])
df_percentages['S/ APAGAMENTO (%)'] = (cross_table['Não']/total) * 100
df_percentages['C/ APAGAMENTO (%)'] = (cross_table['Sim'] / total) * 100
df_percentages

apagamento_plural,Não,Sim,S/ APAGAMENTO (%),C/ APAGAMENTO (%)
curso,,,,
DIREITO,212,41,83.794466,16.205534
LETRAS,228,0,100.000000,0.000000
PEDAGOGIA,259,46,84.918033,15.081967


In [32]:
cross_table = pd.crosstab(df['ano'], df['apagamento_plural'])
df_percentages = cross_table.copy()
total = (cross_table['Sim'] + cross_table['Não'])
df_percentages['S/ APAGAMENTO (%)'] = (cross_table['Não']/total) * 100
df_percentages['C/ APAGAMENTO (%)'] = (cross_table['Sim'] / total) * 100
df_percentages

apagamento_plural,Não,Sim,S/ APAGAMENTO (%),C/ APAGAMENTO (%)
ano,,,,
PRIMEIRO,317,63,83.421053,16.578947
ÚLTIMO,382,24,94.088670,5.911330
